In [8]:
import pandas as pd
import ast
import matplotlib

from itertools import tee, combinations



### Import Data, Convert Durations to Ratios, add to DataFrame

In [9]:
results = pd.read_csv('/Users/rfreedma/Documents/Python_Projects/Dev_Folder/sample_pair.csv')
results.rename(columns=
                   {'Pattern Generating Match': 'Pattern_Generating_Match', 
                    'Pattern matched':'Pattern_Matched',
                    'Piece Title': 'Piece_Title',
                    'First Note Measure Number': 'First_Note_Measure_Number',
                   'Last Note Measure Number': 'Last_Note_Measure_Number',
                    'Note Durations': 'Note_Durations'
                   },
                    inplace=True)
results.head()

,Pattern_Generating_Match,Pattern_Matched,Piece_Title,Part,First_Note_Measure_Number,Last_Note_Measure_Number,Note_Durations,EMA,EMA url
0,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,1,3,"[4.0, 6.0, 2.0, 2.0, 2.0, 2.0, 3.0]","1-3/1/@1.0-end,@start-end,@start-2.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
1,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,10,12,"[4.0, 6.0, 2.0, 2.0, 2.0, 4.0, 2.0]","10-12/1/@1.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
2,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,31,33,"[2.0, 6.0, 2.0, 2.0, 4.0, 2.0, 4.0]","31-33/1/@2.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
3,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Contratenor,34,36,"[2.0, 6.0, 2.0, 2.0, 2.0, 3.0, 1.0]","34-36/2/@4.0-end,@start-end,@start-4.5",https://ema.crimproject.org/https%3A%2F%2Fcrim...
4,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,PrimusTenor,5,7,"[4.0, 6.0, 2.0, 2.0, 2.0, 2.0, 3.0]","5-7/3/@1.0-end,@start-end,@start-2.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...


In [10]:
# converts strings to numbers and replaces these in the dataframe
results['Note_Durations'] = results['Note_Durations'].apply(ast.literal_eval)

durations = results['Note_Durations']

In [11]:
# Create function for pairs and pairs of ratios


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def get_ratios(input_list):
    ratio_pairs = []
    for a, b in pairwise(input_list):
        ratio_pairs.append(b / a)
    return ratio_pairs
   

In [12]:
# 
results["Duration_Ratios"] = results.Note_Durations.apply(get_ratios)
results.head(5)

,Pattern_Generating_Match,Pattern_Matched,Piece_Title,Part,First_Note_Measure_Number,Last_Note_Measure_Number,Note_Durations,EMA,EMA url,Duration_Ratios
0,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,1,3,"[4.0, 6.0, 2.0, 2.0, 2.0, 2.0, 3.0]","1-3/1/@1.0-end,@start-end,@start-2.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,"[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.5]"
1,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,10,12,"[4.0, 6.0, 2.0, 2.0, 2.0, 4.0, 2.0]","10-12/1/@1.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,"[1.5, 0.3333333333333333, 1.0, 1.0, 2.0, 0.5]"
2,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Superius,31,33,"[2.0, 6.0, 2.0, 2.0, 4.0, 2.0, 4.0]","31-33/1/@2.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,"[3.0, 0.3333333333333333, 1.0, 2.0, 0.5, 2.0]"
3,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,Contratenor,34,36,"[2.0, 6.0, 2.0, 2.0, 2.0, 3.0, 1.0]","34-36/2/@4.0-end,@start-end,@start-4.5",https://ema.crimproject.org/https%3A%2F%2Fcrim...,"[3.0, 0.3333333333333333, 1.0, 1.0, 1.5, 0.333..."
4,"[5, -2, 2, 3, -2, -2]","[5, -2, 2, 3, -2, -2]",Veni speciosam,PrimusTenor,5,7,"[4.0, 6.0, 2.0, 2.0, 2.0, 2.0, 3.0]","5-7/3/@1.0-end,@start-end,@start-2.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,"[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.5]"


### Test of Compare Ratios with Sample Data

In [ ]:
def compare_ratios_test (ratios_1, ratios_2):

    # subraction of lists 
    # using zip() + list comprehension 

    # initializing lists 
    ratios_1 = [3, 5, 2, 6, 4] 
    ratios_2 = [7, 3, 4, 1, 5] 

    # printing original lists 
    print ("The original list 1 is : " + str(ratios_1)) 
    print ("The original list 2 is : " + str(ratios_2)) 

    # division of lists 
    # using zip() + list comprehension 
    diffs = [i - j for i, j in zip(ratios_1, ratios_2)] 
    abs_diffs = [abs(ele) for ele in diffs] 
    sum_diffs = sum(abs_diffs)

    # printing result 
    print ("The difference list is : " + str(diffs)) 
    print ("The sum of absolute diffs is : " + str(sum_diffs)) 
    #return abs_diffs
    return sum_diffs
    
    

### And Run the Test

In [ ]:
ratios_1 = [3, 5, 2, 6, 4] 
ratios_2 = [7, 3, 4, 1, 5] 

compare_ratios_test (ratios_1, ratios_2)




### Compare Ratios with Actual Data from CSV

In [ ]:
# Here we group the rows in the DF by the Pattern Generating Match
# Each has its own string of durations, and duration ratios
# and then we compare the ratios to get the differences
# the "list(combinations)" method takes care of building the pairs, using data from our dataframe 'results'

def compare_ratios (ratios_1, ratios_2):
    
    ## division of lists 
    # using zip() + list comprehension 
    diffs = [i - j for i, j in zip(ratios_1, ratios_2)] 
    abs_diffs = [abs(ele) for ele in diffs] 
    sum_diffs = sum(abs_diffs)

    return sum_diffs
    
# this MUST be before the for loop, since it's used there.
matches_1 = []

for name, group in results.groupby("Pattern_Generating_Match"):
    #print(name)
    #print(group)
    ratio_pairs = list(combinations(group.Duration_Ratios.values, 2))
    for a,b in ratio_pairs: 
        sum_diffs = compare_ratios(a,b)
        
        matches_1.append(
            {
                "Pattern_Generating_Match": name,
                "Duration_Ratio_1": a,
                "Duration_Ratio_2": b,
                "Sum_Diffs": sum_diffs
            }
        )


## Use THIS ONE

In [13]:
def compare_ratios_2 (ratios_1, ratios_2):
    
    ## division of lists 
    # using zip() + list comprehension 
    diffs = [i - j for i, j in zip(ratios_1, ratios_2)] 
    abs_diffs = [abs(ele) for ele in diffs] 
    sum_diffs = sum(abs_diffs)

    return sum_diffs

matches_2 = []

for name, group in results.groupby("Pattern_Generating_Match"):
    
    ratio_pairs = list(combinations(group.index.values, 2))
    
    for a,b in ratio_pairs:
        sum_diffs = compare_ratios_2(results.iloc[a].Duration_Ratios, results.iloc[b].Duration_Ratios)
        
        matches_2.append(
            {
                "pattern": name,
                "ratio_1": results.iloc[a].Duration_Ratios,
                "ratio_2": results.iloc[b].Duration_Ratios,
                "sum_diffs": sum_diffs,
                "match_1_index": a,
                "match_2_index": b,
                "match_1_ema": results.iloc[a].EMA,
                "match_2_ema": results.iloc[b].EMA
            }
        )

In [14]:
compare_ratios_2
matches_df=pd.DataFrame(matches_2)
matches_df.head(5)
# could SORT by these!
matches_df[matches_df.sum_diffs == 1]

,pattern,ratio_1,ratio_2,sum_diffs,match_1_index,match_2_index,match_1_ema,match_2_ema
4,"[-2, -2, -2, -2, -2, -2]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 2.0]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 1.0]",1.0,603,608,"74-76/1/@3.0-end,@start-end,@start-1.0","29-31/1/@3.0-end,@start-end,@start-1.0"
63,"[-2, -2, -2, -2, -2, -2]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 2.0]",1.0,605,624,"113-114/1/@2.5-end,@start-1.5","52-53/1/@4.0-end,@start-3.0"
67,"[-2, -2, -2, -2, -2, -2]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 2.0]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 1.0]",1.0,606,608,"80-81/4/@1.0-end,@start-3.0","29-31/1/@3.0-end,@start-end,@start-1.0"
105,"[-2, -2, -2, -2, -2, -2]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 1.0]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 2.0]",1.0,608,609,"29-31/1/@3.0-end,@start-end,@start-1.0","24-25/3/@1.0-end,@start-3.0"
109,"[-2, -2, -2, -2, -2, -2]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 1.0]","[0.5, 1.5, 0.3333333333333333, 1.0, 1.0, 2.0]",1.0,608,613,"29-31/1/@3.0-end,@start-end,@start-1.0","63-65/1/@3.0-end,@start-end,@start-1.0"
...,...,...,...,...,...,...,...,...
12930,"[5, -2, 2, 3, -2, -2]","[1.5, 0.3333333333333333, 1.0, 1.0, 0.5, 1.0]","[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.5]",1.0,13,24,"1-3/3/@1.0-end,@start-end,@start-1.5","1-3/4/@1.0-end,@start-end,@start-2.0"
12945,"[5, -2, 2, 3, -2, -2]","[1.5, 0.3333333333333333, 1.0, 1.0, 2.0, 1.0]","[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0]",1.0,15,18,"2-4/1/@1.0-end,@start-end,@start-3.0","103-105/3/@1.0-end,@start-end,@start-2.0"
12947,"[5, -2, 2, 3, -2, -2]","[1.5, 0.3333333333333333, 1.0, 1.0, 2.0, 1.0]","[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0]",1.0,15,20,"2-4/1/@1.0-end,@start-end,@start-3.0","9-11/4/@1.0-end,@start-end,@start-2.0"
12962,"[5, -2, 2, 3, -2, -2]","[1.5, 0.3333333333333333, 1.0, 1.0, 2.0, 1.0]","[1.5, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0]",1.0,17,18,"6-8/3/@1.0-end,@start-end,@start-3.0","103-105/3/@1.0-end,@start-end,@start-2.0"


In [ ]:
matches_df=pd.DataFrame(matches)
# could SORT by these!
matches_df[matches_df.Sum_Diffs == 2]

In [ ]:
super_matches.plot(kind="hist")

In [ ]:
matches_df.sort_values(["sum_diffs", "Pattern_Generating_Match"], ascending=False)

In [ ]:
matches_df.Sum_Diffs.hist()

In [ ]:
plotdata = matches_df
plotdata.plot(kind="bar")

In [ ]:
matches_df['Part'] = results.Part
matches_df['Piece_Title'] = results.Piece_Title
matches_df.head()



In [ ]:
matches_df.Part.hist()

In [ ]:
matches_df.Piece_Title.hist()

### Test Data

In [ ]:
ratios_1 = [3, 5, 2, 6, 4] 
ratios_2 = [7, 3, 4, 1, 5] 


compare_ratios(ratios_1, ratios_2)